In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os

In [14]:
from var_func import *

# Loading Data

## Konsumsi Antara

In [2]:
data_path = 'G:\My Drive\STIS\Skripsi\data\Input Output'

In [4]:
konsumsi_akhir = pd.read_excel(
    os.path.join(data_path, 'Databases2.xlsx'),
    sheet_name = 'Konsumsi Akhir'
)

In [12]:
konsumsi_akhir

,Provinsi Penyedia,Industri Penyedia,Provinsi Pengguna,Komponen Pengeluaran,Nilai Juta
0,11,I-01,11,3011,2.249330e+05
1,11,I-01,11,3012,0.000000e+00
2,11,I-01,11,3020,0.000000e+00
3,11,I-01,11,3030,0.000000e+00
4,11,I-01,11,3041,1.383529e+05
...,...,...,...,...,...
300555,94,I-52,94,3011,1.672438e+06
300556,94,I-52,94,3012,2.222569e+06
300557,94,I-52,94,3020,1.240059e+05
300558,94,I-52,94,3030,1.553437e+05


## Input Antara Nasional

In [35]:
data_path = 'G:\My Drive\STIS\Skripsi\data\Input Output'
data = pd.read_excel(
    os.path.join(data_path, 'Databases2.xlsx'),
    header = None,
    sheet_name = 'Input Antara Nasional'
)

data[0] = data[0].ffill()
data = data.transpose()
data[0] = data[0].ffill()
data = data.transpose()

column = pd.MultiIndex.from_arrays(data.iloc[0:2,2:].astype(str).to_numpy())
row = pd.MultiIndex.from_arrays(data.iloc[2:,0:2].transpose().to_numpy())
x = pd.DataFrame(
    data.iloc[2:,2:].to_numpy(),
    columns = column,
    index = row
)

c = x.reset_index().melt(id_vars = ['level_0', 'level_1']).astype({
    'level_0':str,
    'level_1':str,
    'variable_0':str,
    'variable_1':str,
    'value':float,
})

test = pd.read_excel(
    os.path.join(data_path, 'Databases2.xlsx'),\
    sheet_name = 'Test'
)

x = c.join(test.set_index('test'), on = 'level_0')[['Kode', 'level_1', 'variable_0', 'variable_1', 'value']].rename(columns = {
    'Kode':'Provinsi Penyedia',
    'level_1':'Industri Penyedia',
    'variable_0':'a',
    'variable_1':'Industri Pengguna',
    'value':'Nilai Juta'
}).join(test.set_index('test'), on = 'a')[['Provinsi Penyedia', 'Industri Penyedia', 'Kode', 'Industri Pengguna', 'Nilai Juta']].rename(columns = {
    'Kode':'Provinsi Pengguna'
}).sort_values(['Provinsi Penyedia', 'Industri Penyedia', 'Provinsi Pengguna', 'Industri Pengguna']).reset_index(drop = True).astype({
    'Provinsi Penyedia':str,
    'Industri Penyedia':str,
    'Provinsi Pengguna':str,
    'Industri Pengguna':str,
    'Nilai Juta':float
})

In [49]:
table_name = 'input_antara_nasional'
tuple_list = []

sql = f'''
INSERT INTO {table_name}
VALUES (%s, %s, %s, %s, %s)
'''

for val in x.values:
    tuple_list.append(tuple(val))

In [ ]:
conn = get_connection()
cur = conn.cursor()

cur.executemany(sql, tuple_list)

conn.commit()
cur.close()
conn.close()

In [ ]:
'a'

## Kode Provinsi

In [ ]:
kode_prov = pd.read_excel(
    os.path.join(data_path, 'Databases2.xlsx'),
    sheet_name = 'Kode Provinsi'
)
kode_pend = pd.read_excel(
    os.path.join(data_path, 'Databases2.xlsx'),
    sheet_name = 'Kode Pendapatan'
)
kode_ind = pd.read_excel(
    os.path.join(data_path, 'Databases2.xlsx'),
    sheet_name = 'Kode Industri'
)


# Inserting Data

In [19]:
data_path = 'G:\My Drive\STIS\Skripsi\data\Input Output\IRIO'

In [33]:
sheet_name = 'Kode Lapus'
table_name = 'kode_lapus'
tuple_list = []

data = pd.read_excel(
    os.path.join(data_path, f'{sheet_name}.xlsx')
)

sql = f'''
INSERT INTO {table_name}
VALUES (%s, %s)
'''

for val in data.values:
    tuple_list.append(tuple(val))

In [34]:
conn = get_connection()
cur = conn.cursor()

cur.executemany(sql, tuple_list)

conn.commit()
cur.close()
conn.close()

<module 'psycopg2.sql' from 'C:\\Users\\User\\anaconda3\\lib\\site-packages\\psycopg2\\sql.py'>